In [1]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
import string
import sklearn
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import seaborn as sns

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble,calibration

#from sklearn.cross_validation import KFold
#from sklearn.cross_validation import train_test_split

#import pandas, numpy, textblob, string
#from keras.preprocessing import text, sequence
#from keras import layers, models, optimizers

In [2]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

train = pd.read_csv(r'training_matrix.csv')
valid = pd.read_csv(r'validating_matrix.csv')
test = pd.read_csv(r'testing_matrix.csv')

train.loc[train.label == 1, 'label'] = 0
train.loc[train.label == -1, 'label'] = 1

valid.loc[valid.label == 1, 'label'] = 0
valid.loc[valid.label == -1, 'label'] = 1

test.loc[test.label == 1, 'label'] = 0
test.loc[test.label == -1, 'label'] = 1


train['char_count'] = train['review'].apply(len)
train['word_count'] = train['review'].apply(lambda x: len(x.split()))
train['word_density'] = train['char_count'] / (train['word_count']+1)
train['punctuation_count'] = train['review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
train['title_word_count'] = train['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
train['upper_case_word_count'] = train['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

valid['char_count'] = valid['review'].apply(len)
valid['word_count'] = valid['review'].apply(lambda x: len(x.split()))
valid['word_density'] = valid['char_count'] / (valid['word_count']+1)
valid['punctuation_count'] = valid['review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
valid['title_word_count'] = valid['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
valid['upper_case_word_count'] = valid['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

test['char_count'] = test['review'].apply(len)
test['word_count'] = test['review'].apply(lambda x: len(x.split()))
test['word_density'] = test['char_count'] / (test['word_count']+1)
test['punctuation_count'] = test['review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
test['title_word_count'] = test['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
test['upper_case_word_count'] = test['review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Turker\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# reviewer_centric features #
#train.user_id.value_counts()

train['date.1'] = pd.to_datetime(train['date.1'])
train['day_of_week'] = train['date.1'].dt.day_name()
#train.day_of_week.value_counts()[:10]]
dofw = train.day_of_week.unique()
for day in range(len(dofw)):
        label=train['label'][train.day_of_week == dofw[day]]
        fake=len(label[label==1])
        real=len(label[label==0])
        
        print("***********Day is ",dofw[day])
        print("fake is ",fake)
        print("real is ",real)
        print("tot is ",fake+real)
        print("Ratio is ",100*(fake/(fake+real)))
        print("len is ",len(label))

train['user_id_no_of_review'] = train.groupby('user_id')['user_id'].transform('size')
train['user_id_ave_rating'] = train.groupby('user_id')['rating'].transform('mean')
#train['user_id_std_rating'] = train.groupby('user_id')['rating'].transform('std')
train['user_id_ave_no_words'] = train.groupby('user_id')['word_count'].transform('mean')
train['user_id_max_review_a_day'] = train['user_id_no_of_review']
grouped = train.groupby('user_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    train.loc[train.user_id == name,'user_id_max_review_a_day'] = df2
    #group['user_id_max_review_a_day']=df2

#train[:][train.user_id==9236].sort_index(axis = 0)
    
valid['date.1'] = pd.to_datetime(valid['date.1'])
valid['day_of_week'] = valid['date.1'].dt.day_name()
#valid.day_of_week.value_counts()[:10]]
dofw = valid.day_of_week.unique()
for day in range(len(dofw)):
        label=valid['label'][valid.day_of_week == dofw[day]]
        fake=len(label[label==1])
        real=len(label[label==0])
        
        print("***********Day is ",dofw[day])
        print("fake is ",fake)
        print("real is ",real)
        print("tot is ",fake+real)
        print("Ratio is ",100*(fake/(fake+real)))
        print("len is ",len(label))

valid['user_id_no_of_review'] = valid.groupby('user_id')['user_id'].transform('size')
valid['user_id_ave_rating'] = valid.groupby('user_id')['rating'].transform('mean')
#valid['user_id_std_rating'] = valid.groupby('user_id')['rating'].transform('std')
valid['user_id_ave_no_words'] = valid.groupby('user_id')['word_count'].transform('mean')
valid['user_id_max_review_a_day'] = valid['user_id_no_of_review']
grouped = valid.groupby('user_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    valid.loc[valid.user_id == name,'user_id_max_review_a_day'] = df2
    #group['user_id_max_review_a_day']=df2

#valid[:][valid.user_id==9236].sort_index(axis = 0) 

test['date.1'] = pd.to_datetime(test['date.1'])
test['day_of_week'] = test['date.1'].dt.day_name()
#test.day_of_week.value_counts()[:10]]
dofw = test.day_of_week.unique()
for day in range(len(dofw)):
        label=test['label'][test.day_of_week == dofw[day]]
        fake=len(label[label==1])
        real=len(label[label==0])
        
        print("***********Day is ",dofw[day])
        print("fake is ",fake)
        print("real is ",real)
        print("tot is ",fake+real)
        print("Ratio is ",100*(fake/(fake+real)))
        print("len is ",len(label))

test['user_id_no_of_review'] = test.groupby('user_id')['user_id'].transform('size')
test['user_id_ave_rating'] = test.groupby('user_id')['rating'].transform('mean')
#test['user_id_std_rating'] = test.groupby('user_id')['rating'].transform('std')
test['user_id_ave_no_words'] = test.groupby('user_id')['word_count'].transform('mean')
test['user_id_max_review_a_day'] = test['user_id_no_of_review']
grouped = test.groupby('user_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    test.loc[test.user_id == name,'user_id_max_review_a_day'] = df2
    #group['user_id_max_review_a_day']=df2

#test[:][test.user_id==9236].sort_index(axis = 0)   

***********Day is  Thursday
fake is  764
real is  755
tot is  1519
Ratio is  50.29624753127058
len is  1519
***********Day is  Tuesday
fake is  800
real is  788
tot is  1588
Ratio is  50.377833753148614
len is  1588
***********Day is  Friday
fake is  758
real is  701
tot is  1459
Ratio is  51.953392734749826
len is  1459
***********Day is  Saturday
fake is  756
real is  744
tot is  1500
Ratio is  50.4
len is  1500
***********Day is  Monday
fake is  858
real is  913
tot is  1771
Ratio is  48.4472049689441
len is  1771
***********Day is  Sunday
fake is  790
real is  903
tot is  1693
Ratio is  46.66272888363851
len is  1693
***********Day is  Wednesday
fake is  836
real is  758
tot is  1594
Ratio is  52.446675031367626
len is  1594
***********Day is  Tuesday
fake is  150
real is  161
tot is  311
Ratio is  48.231511254019296
len is  311
***********Day is  Thursday
fake is  129
real is  128
tot is  257
Ratio is  50.19455252918288
len is  257
***********Day is  Saturday
fake is  137
real is 

In [4]:
# product_centric features #
#train.prod_id.value_counts()

train['prod_id_no_of_review'] = train.groupby('prod_id')['prod_id'].transform('size') + 1000.
train['prod_id_ave_rating'] = train.groupby('prod_id')['rating'].transform('mean') + 1000.
#train['prod_id_std_rating'] = train.groupby('prod_id')['rating'].transform('std') + 1000.
train['prod_id_ave_no_words'] = train.groupby('prod_id')['word_count'].transform('mean') + 1000.
train['prod_id_max_review_a_day'] = train['prod_id_no_of_review'] + 1000.
grouped = train.groupby('prod_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    train.loc[train.prod_id == name,'prod_id_max_review_a_day'] = df2 + 1000.
    #group['prod_id_max_review_a_day']=df2

#train[:][train.prod_id==9236].sort_index(axis = 0)

valid['prod_id_no_of_review'] = valid.groupby('prod_id')['prod_id'].transform('size') + 1000.
valid['prod_id_ave_rating'] = valid.groupby('prod_id')['rating'].transform('mean') + 1000.
#valid['prod_id_std_rating'] = valid.groupby('prod_id')['rating'].transform('std') + 1000.
valid['prod_id_ave_no_words'] = valid.groupby('prod_id')['word_count'].transform('mean') + 1000.
valid['prod_id_max_review_a_day'] = valid['prod_id_no_of_review']
grouped = valid.groupby('prod_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    valid.loc[valid.prod_id == name,'prod_id_max_review_a_day'] = df2 + 1000.
    #group['prod_id_max_review_a_day']=df2

#valid[:][valid.prod_id==9236].sort_index(axis = 0)  

test['prod_id_no_of_review'] = test.groupby('prod_id')['prod_id'].transform('size') + 1000.
test['prod_id_ave_rating'] = test.groupby('prod_id')['rating'].transform('mean') + 1000.
#test['prod_id_std_rating'] = test.groupby('prod_id')['rating'].transform('std') + 1000.
test['prod_id_ave_no_words'] = test.groupby('prod_id')['word_count'].transform('mean') + 1000.
test['prod_id_max_review_a_day'] = test['prod_id_no_of_review']
grouped = test.groupby('prod_id')

for name,group in grouped:
    #print(name)
    #print(group)
    df2 = group.groupby('date').size().max()
    #print(df2)
    test.loc[test.prod_id == name,'prod_id_max_review_a_day'] = df2 + 1000.
    #group['prod_id_max_review_a_day']=df2

#test[:][test.prod_id==9236].sort_index(axis = 0)  
                  

In [5]:
train['total']=train.user_id.astype(str) + ' '+ train.prod_id.astype(str) + ' ' + train.rating.astype(str) + ' '+ \
train.user_id_no_of_review.astype(str) + ' ' + train.user_id_ave_rating.astype(str) + ' '+\
train.user_id_ave_no_words.astype(str) + ' ' + train.user_id_max_review_a_day.astype(str) + ' '+\
train.prod_id_no_of_review.astype(str) + ' ' + train.prod_id_ave_rating.astype(str) + ' '+\
train.prod_id_ave_no_words.astype(str) + ' ' + train.prod_id_max_review_a_day.astype(str) + ' '+\
train.day_of_week.astype(str) + ' '+train.review  

#train['total']=train.user_id.astype(str) + ' '+ train.prod_id.astype(str) + ' ' + train.rating.astype(str) + ' '+  train.review
#test['total']=test.user_id.astype(str) + ' '+ test.prod_id.astype(str) + ' ' + test.rating.astype(str) + ' '+test.review

valid['total']=valid.user_id.astype(str) + ' '+ valid.prod_id.astype(str) + ' ' + valid.rating.astype(str) + ' '+ \
valid.user_id_no_of_review.astype(str) + ' ' + valid.user_id_ave_rating.astype(str) + ' '+\
valid.user_id_ave_no_words.astype(str) + ' ' + valid.user_id_max_review_a_day.astype(str) + ' '+\
valid.prod_id_no_of_review.astype(str) + ' ' + valid.prod_id_ave_rating.astype(str) + ' '+\
valid.prod_id_ave_no_words.astype(str) + ' ' + valid.prod_id_max_review_a_day.astype(str) + ' '+\
valid.day_of_week.astype(str) + ' '+ valid.review  

test['total']=test.user_id.astype(str) + ' '+ test.prod_id.astype(str) + ' ' + test.rating.astype(str) + ' '+ \
test.user_id_no_of_review.astype(str) + ' ' + test.user_id_ave_rating.astype(str) + ' '+\
test.user_id_ave_no_words.astype(str) + ' ' + test.user_id_max_review_a_day.astype(str) + ' '+\
test.prod_id_no_of_review.astype(str) + ' ' + test.prod_id_ave_rating.astype(str) + ' '+\
test.prod_id_ave_no_words.astype(str) + ' ' + test.prod_id_max_review_a_day.astype(str) + ' '+\
test.day_of_week.astype(str) + ' '+ test.review  

In [6]:
from nltk import SnowballStemmer
porter = SnowballStemmer("english")

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Turker\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
from sklearn.decomposition import PCA
from scipy import sparse
def apply_PCA(feature_vector_train, feature_vector_valid, feature_vector_test, n_components = 300):
    # applys PCA
    pca = PCA(n_components)
    #pca = PCA(0.95)
    pca.fit(feature_vector_train.toarray())
    xtrain = pca.transform(feature_vector_train.toarray())  
    xvalid = pca.transform(feature_vector_valid.toarray())       
    xtest  = pca.transform(feature_vector_test.toarray()) 
    
    feature_vector_train = sparse.csr_matrix(xtrain)
    feature_vector_valid = sparse.csr_matrix(xvalid)
    feature_vector_test  = sparse.csr_matrix(xtest)    
    
    return [feature_vector_train, feature_vector_valid, feature_vector_test]

from sklearn.decomposition import LatentDirichletAllocation
def apply_LDA(feature_vector_train, feature_vector_valid, feature_vector_test, n_components = 300):
    # applys lda
    lda = LatentDirichletAllocation(n_components, random_state=0)
    lda.fit(feature_vector_train.toarray())
    xtrain = lda.transform(feature_vector_train.toarray())  
    xvalid = lda.transform(feature_vector_valid.toarray())       
    xtest  = lda.transform(feature_vector_test.toarray()) 
    
    feature_vector_train = sparse.csr_matrix(xtrain)
    feature_vector_valid = sparse.csr_matrix(xvalid)
    feature_vector_test  = sparse.csr_matrix(xtest)    
    
    return [feature_vector_train, feature_vector_valid, feature_vector_test]


def scale_feature(col):
    x = col.values.astype(float)
    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler()    
    # Create an object to transform the data to fit minmax processor
    x_scaled = min_max_scaler.fit_transform(x)
    col = x_scaled
    return col




from scipy.sparse import hstack
def add_features(feature_vector_train, feature_vector_valid, feature_vector_test):
    
    feature_vector_train = hstack((feature_vector_train,np.array(train['user_id_no_of_review'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['user_id_ave_rating'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['user_id_ave_no_words'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['user_id_max_review_a_day'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['prod_id_no_of_review'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['prod_id_ave_rating'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['prod_id_ave_no_words'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['prod_id_max_review_a_day'])[:,None]))    
    feature_vector_train = hstack((feature_vector_train,np.array(train['char_count'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['word_count'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['word_density'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['punctuation_count'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['title_word_count'])[:,None]))
    feature_vector_train = hstack((feature_vector_train,np.array(train['upper_case_word_count'])[:,None]))
    
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['user_id_no_of_review'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['user_id_ave_rating'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['user_id_ave_no_words'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['user_id_max_review_a_day'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['prod_id_no_of_review'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['prod_id_ave_rating'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['prod_id_ave_no_words'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['prod_id_max_review_a_day'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['char_count'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['word_count'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['word_density'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['punctuation_count'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['title_word_count'])[:,None]))
    feature_vector_valid = hstack((feature_vector_valid,np.array(valid['upper_case_word_count'])[:,None]))
    
    
    feature_vector_test = hstack((feature_vector_test,np.array(test['user_id_no_of_review'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['user_id_ave_rating'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['user_id_ave_no_words'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['user_id_max_review_a_day'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['prod_id_no_of_review'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['prod_id_ave_rating'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['prod_id_ave_no_words'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['prod_id_max_review_a_day'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['char_count'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['word_count'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['word_density'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['punctuation_count'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['title_word_count'])[:,None]))
    feature_vector_test = hstack((feature_vector_test,np.array(test['upper_case_word_count'])[:,None]))
    
    return [feature_vector_train, feature_vector_valid, feature_vector_test]

In [8]:
# create a count vectorizer object 
#count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000, stop_words='english')
#count_vect.fit(train.total)
#
#x_train_count =  count_vect.transform(train.total)
#x_valid_count =  count_vect.transform(valid.total)
#x_test_count =  count_vect.transform(test.total)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,2), max_features=5000, stop_words='english',min_df=5)
tfidf_vect_ngram.fit(train.review)
x_train_tfidf_ngram =  tfidf_vect_ngram.transform(train.review)
x_valid_tfidf_ngram =  tfidf_vect_ngram.transform(valid.review)
x_test_tfidf_ngram =  tfidf_vect_ngram.transform(test.review)


# ngram level tf-idf 
tfidf_vect_ngram_NB = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,2), max_features=5000, stop_words='english',min_df=5)
tfidf_vect_ngram_NB.fit(train.total)
x_train_tfidf_ngram_NB =  tfidf_vect_ngram_NB.transform(train.total)
x_valid_tfidf_ngram_NB =  tfidf_vect_ngram_NB.transform(valid.total)
x_test_tfidf_ngram_NB =  tfidf_vect_ngram_NB.transform(test.total)
#[x_train_tfidf_ngram, x_test_tfidf_ngram] = apply_PCA(x_train_tfidf_ngram, x_test_tfidf_ngram)
#[x_train_tfidf_ngram, x_test_tfidf_ngram] = add_features(x_train_tfidf_ngram, x_test_tfidf_ngram)



def train_model(classifier, feature_vector_train, label, feature_vector_valid,label_valid, is_neural_net=False):
    # fit the train dataset on the classifier
    classifier.fit(feature_vector_train, label)
    # predict the labels on validation dataset
    probs = classifier.predict_proba(feature_vector_valid)
    #print("Probs is ",probs[:,1])
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    return [predictions,metrics.accuracy_score(predictions, label_valid),probs]

## Create first pipeline for base without reducing features.
#from sklearn.pipeline import Pipeline
#from sklearn.model_selection import GridSearchCV
#
#pipe = Pipeline([('classifier' , linear_model.LogisticRegression())])
## pipe = Pipeline([('classifier', RandomForestClassifier())])
##lr = linear_model.LogisticRegression()

In [9]:
train_y=train.label
test_y=test.label
valid_y=valid.label


train_x=x_train_tfidf_ngram
test_x=x_test_tfidf_ngram
valid_x=x_valid_tfidf_ngram
print("************ NB Test***********")

[predictions,accuracy,probs] = train_model(naive_bayes.MultinomialNB(), train_x, train_y, test_x, test_y)
print ("Accuracy is  ", accuracy)
fpr, tpr, thresholds = metrics.roc_curve(test_y, probs[:,1])
auc = metrics.roc_auc_score(test_y, probs[:,1])
print("Auc ROC is ",auc)
precision, recall, thresholds = metrics.precision_recall_curve(test_y, probs[:,1])
auc = metrics.auc(recall, precision)
print("Auc PRC is ",auc)
f1 = metrics.f1_score(test_y, predictions)
print("F1 score is",f1)
print("Log loss is ",metrics.log_loss(test_y,probs[:,1]))

print("************ NB Valid***********")

[predictions,accuracy,probs] = train_model(naive_bayes.MultinomialNB(), train_x, train_y, valid_x, valid_y)
print ("Accuracy is  ", accuracy)
fpr, tpr, thresholds = metrics.roc_curve(valid_y, probs[:,1])
auc = metrics.roc_auc_score(valid_y, probs[:,1])
print("Auc ROC is ",auc)
precision, recall, thresholds = metrics.precision_recall_curve(valid_y, probs[:,1])
auc = metrics.auc(recall, precision)
print("Auc PRC is ",auc)
f1 = metrics.f1_score(valid_y, predictions)
print("F1 score is",f1)
print("Log loss is ",metrics.log_loss(valid_y,probs[:,1]))

print("************ NB Train***********")

[predictions,accuracy,probs] = train_model(naive_bayes.MultinomialNB(), train_x, train_y, train_x, train_y)
print ("Accuracy is  ", accuracy)
fpr, tpr, thresholds = metrics.roc_curve(train_y, probs[:,1])
auc = metrics.roc_auc_score(train_y, probs[:,1])
print("Auc ROC is ",auc)
precision, recall, thresholds = metrics.precision_recall_curve(train_y, probs[:,1])
auc = metrics.auc(recall, precision)
print("Auc PRC is ",auc)
f1 = metrics.f1_score(train_y, predictions)
print("F1 score is",f1)
print("Log loss is ",metrics.log_loss(train_y,probs[:,1]))

************ NB Test***********
Accuracy is   0.6386
Auc ROC is  0.69255616
Auc PRC is  0.6647269111392298
F1 score is 0.65396399846802
Log loss is  0.6335912741332487
************ NB Valid***********
Accuracy is   0.657
Auc ROC is  0.703815
Auc PRC is  0.6764445297129313
F1 score is 0.6669902912621359
Log loss is  0.6282846644737022
************ NB Train***********
Accuracy is   0.7304027328299173
Auc ROC is  0.8016306262728762
Auc PRC is  0.786762282790817
F1 score is 0.7336826214368173
Log loss is  0.568375774958194


In [10]:
train_x=x_train_tfidf_ngram_NB
test_x=x_test_tfidf_ngram_NB
valid_x=x_valid_tfidf_ngram_NB
print("************ NB Test***********")

[predictions,accuracy,probs] = train_model(naive_bayes.MultinomialNB(), train_x, train_y, test_x, test_y)
print ("Accuracy is  ", accuracy)
fpr, tpr, thresholds = metrics.roc_curve(test_y, probs[:,1])
auc = metrics.roc_auc_score(test_y, probs[:,1])
print("Auc ROC is ",auc)
precision, recall, thresholds = metrics.precision_recall_curve(test_y, probs[:,1])
auc = metrics.auc(recall, precision)
print("Auc PRC is ",auc)
f1 = metrics.f1_score(test_y, predictions)
print("F1 score is",f1)
print("Log loss is ",metrics.log_loss(test_y,probs[:,1]))

recall_score = metrics.recall_score(test_y, predictions)
print("recall_score score is",recall_score)
precision_score = metrics.precision_score(test_y, predictions)
print("precision_score score is",precision_score)


results = pd.DataFrame()
results['prob'] = probs[:,1]
results['predictions'] = predictions
results['labels'] = test_y
results.to_csv(('NB.csv'))


print("************ NB Valid***********")

[predictions,accuracy,probs] = train_model(naive_bayes.MultinomialNB(), train_x, train_y, valid_x, valid_y)
print ("Accuracy is  ", accuracy)
fpr, tpr, thresholds = metrics.roc_curve(valid_y, probs[:,1])
auc = metrics.roc_auc_score(valid_y, probs[:,1])
print("Auc ROC is ",auc)
precision, recall, thresholds = metrics.precision_recall_curve(valid_y, probs[:,1])
auc = metrics.auc(recall, precision)
print("Auc PRC is ",auc)
f1 = metrics.f1_score(valid_y, predictions)
print("F1 score is",f1)
print("Log loss is ",metrics.log_loss(valid_y,probs[:,1]))

print("************ NB Train***********")

[predictions,accuracy,probs] = train_model(naive_bayes.MultinomialNB(), train_x, train_y, train_x, train_y)
print ("Accuracy is  ", accuracy)
fpr, tpr, thresholds = metrics.roc_curve(train_y, probs[:,1])
auc = metrics.roc_auc_score(train_y, probs[:,1])
print("Auc ROC is ",auc)
precision, recall, thresholds = metrics.precision_recall_curve(train_y, probs[:,1])
auc = metrics.auc(recall, precision)
print("Auc PRC is ",auc)
f1 = metrics.f1_score(train_y, predictions)
print("F1 score is",f1)
print("Log loss is ",metrics.log_loss(train_y,probs[:,1]))


************ NB Test***********
Accuracy is   0.6544
Auc ROC is  0.7080433199999999
Auc PRC is  0.6695996516929208
F1 score is 0.680709534368071
Log loss is  0.6241664904935046
recall_score score is 0.7368
precision_score score is 0.632554945054945
************ NB Valid***********
Accuracy is   0.657
Auc ROC is  0.71219
Auc PRC is  0.6841077278045159
F1 score is 0.6815227483751161
Log loss is  0.6222970418207876
************ NB Train***********
Accuracy is   0.7088277597986335
Auc ROC is  0.7765207246325654
Auc PRC is  0.7582896850135389
F1 score is 0.7164492690186466
Log loss is  0.5695794416704953
